In [4]:
%pip install --upgrade --force-reinstall --no-cache-dir ../../

Processing /home/sbumseo/git/ivete
  ERROR: Command errored out with exit status 1:
   command: /home/sbumseo/git/ivete/experiments/qa/.venv/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-n3ykveu1/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-n3ykveu1/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-ui0ad33r
       cwd: /tmp/pip-req-build-n3ykveu1/
  Complete output (6 lines):
  usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: setup.py --help [cmd1 cmd2 ...]
     or: setup.py --help-commands
     or: setup.py cmd --help
  
  error: invalid command 'bdist_wheel'
  ----------------------------------------
  ERROR: Failed building wheel for ivete
  Running setup.py clean for ivete
Failed to build ivete
  Attempting uninstall: ivete
    Found existing installatio

In [5]:
import torch
from transformers import BertConfig, BertForQuestionAnswering, BertTokenizer
from ivete.deploy import DeployArguments, deploy

In [6]:
args = DeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_path="./checkpoints/",
    inference_args={
        'max_seq_length': 128,
        'max_query_length': 32,
    }
)

downstream_model_path: ./checkpoints/epoch=0-val_loss=0.47.ckpt


In [7]:
checkpoint = torch.load(
    args.downstream_model_path,
    map_location=torch.device("cpu"),
)

In [8]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name
)
model = BertForQuestionAnswering(pretrained_model_config)

In [9]:
model.load_state_dict({k.replace("model.", ""): v for k, v in checkpoint['state_dict'].items()})

<All keys matched successfully>

In [10]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [11]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [12]:
def inference(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=args.inference_args['max_query_length'],
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=args.inference_args['max_seq_length'],
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [13]:
context = "한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다."
question = "한강대교가 위치한 곳은?"
inference(question=question, context=context)

{'question': '한강대교가 위치한 곳은?',
 'context': '한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다.',
 'answer': '서울특별시 용산구 이촌동에'}

In [14]:
import os, ivete
os.path.dirname(ivete.__file__)

'/home/sbumseo/git/ivete/experiments/qa/.venv/lib/python3.8/site-packages/ivete'

In [17]:
import os
path = os.path.abspath("../../templates/")
app = deploy(
    inference=inference,
    api_name="qa",
    template_name="default",
    template_path=path,
)
app.run()

 * Serving Flask app 'ivete.deploy' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [20/Jul/2022 10:47:06] "POST /qa/api HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 10:47:10] "GET /qa HTTP/1.1" 308 -
127.0.0.1 - - [20/Jul/2022 10:47:10] "GET /qa/ HTTP/1.1" 200 -
